In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 81)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

import sys

RANDOM_SEED = 13

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras import backend as K
import tensorflow_hub as hub

from datetime import datetime

import sys
from tqdm import tqdm

import zipfile 
#from fastparquet import ParquetFile

print('Python       :', sys.version.split('\n')[0])
print('Pandas       :', pd.__version__)
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

def f1(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return 2 * (K.sum(y_true * y_pred)+ K.epsilon()) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())

Python       : 3.6.9 (default, Oct  8 2020, 12:12:24) 
Pandas       : 1.1.5
Numpy        : 1.19.5
Tensorflow   : 2.4.1


In [ ]:
#train = ParquetFile("/content/drive/My Drive/VTB/data_fusion_train.parquet").to_pandas()
train = pd.read_parquet('/content/drive/My Drive/VTB/data_fusion_train.parquet', use_threads=False)

In [ ]:
X_train = train[train['category_id'] != -1].drop_duplicates('item_name')['item_name']
y_train = train[train['category_id'] != -1].drop_duplicates('item_name')['category_id']

train_data = pd.get_dummies(y_train)

In [ ]:
from sklearn.model_selection import train_test_split

Z_train, Z_test, s_train, s_test = train_test_split(X_train, train_data, test_size=0.13, random_state=42)

In [ ]:
tf.keras.backend.clear_session()
input_1 = Input(shape=(), dtype=tf.string,  name = "Input_1")
#embedding = './nnlm-zh-dim50_2/'

embedding = 'https://tfhub.dev/google/universal-sentence-encoder/4'
hub_layer_1 = hub.KerasLayer(embedding, input_shape=[],  dtype=tf.string, trainable=True)

x = hub_layer_1(input_1)
x = Dense(128, activation="elu")(x)
predictions = Dense(96, activation="softmax")(x)

model = Model(inputs=input_1, outputs=predictions)
model.compile(optimizer='adam', loss='poisson', metrics=['accuracy', f1])
model.summary()  

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_1 (InputLayer)         [(None,)]                 0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 512)               256797824 
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dense_1 (Dense)              (None, 96)                12384     
Total params: 256,875,872
Trainable params: 256,875,872
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(Z_train,
                    s_train,
                    epochs=10,
                    batch_size=500,
                    validation_data = (Z_test, s_test),
                    shuffle=True
                    )

Epoch 1/10
84/84 [==============================] - 19s 172ms/step - loss: 0.0517 - accuracy: 0.1385 - f1: 0.0338 - val_loss: 0.0460 - val_accuracy: 0.1952 - val_f1: 0.0702
Epoch 2/10
84/84 [==============================] - 14s 165ms/step - loss: 0.0444 - accuracy: 0.2397 - f1: 0.0947 - val_loss: 0.0380 - val_accuracy: 0.3872 - val_f1: 0.2325
Epoch 3/10
84/84 [==============================] - 14s 166ms/step - loss: 0.0346 - accuracy: 0.4526 - f1: 0.2965 - val_loss: 0.0325 - val_accuracy: 0.5102 - val_f1: 0.3748
Epoch 4/10
84/84 [==============================] - 14s 165ms/step - loss: 0.0259 - accuracy: 0.6493 - f1: 0.4866 - val_loss: 0.0315 - val_accuracy: 0.5641 - val_f1: 0.4639
Epoch 5/10
84/84 [==============================] - 14s 166ms/step - loss: 0.0194 - accuracy: 0.8008 - f1: 0.6485 - val_loss: 0.0327 - val_accuracy: 0.5923 - val_f1: 0.5144
Epoch 6/10
84/84 [==============================] - 14s 165ms/step - loss: 0.0153 - accuracy: 0.9040 - f1: 0.7777 - val_loss: 0.0346 - 

In [ ]:
prdicty = model.predict(Z_test)

In [ ]:
a = [train_data.columns[i] for i in np.argmax(prdicty, -1)]

In [ ]:
b1 = np.argmax(s_test.values, -1)
b = [train_data.columns[i] for i in b1]

In [ ]:
from sklearn.metrics import f1_score

f1_score(b, a, average='weighted')

0.6243541859825541

In [ ]:
a == b

False

In [ ]:
np.array(a).shape

(15915,)

In [ ]:
train_data.columns[np.argmax(prdicty, -1)]

83

In [ ]:
s_test

,0,1,2,3,4,6,7,9,11,12,13,19,20,24,26,27,29,30,31,35,36,37,38,39,40,41,42,43,45,46,49,50,51,52,53,54,55,56,57,58,...,79,80,81,82,83,84,85,90,92,96,97,100,101,102,103,105,106,107,108,109,111,114,115,117,118,120,128,130,133,138,139,140,145,150,163,164,167,177,203,204
491933,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
821091,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
42108117,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2179229,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3740609,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4154422,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6336466,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
364351,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1766723,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data.columns[np.argmax(s_test)]

83

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
tf.keras.backend.clear_session()
text_input = Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3', name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3', trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
predictions = Dense(96, activation='softmax')(net)

model1 = Model(inputs=text_input, outputs=predictions)
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', f1])
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_word_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'encoder_outputs':  109482241   preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
______________________________________________________________________________________________

In [ ]:
history = model1.fit(X_train,
                    train_data,
                    epochs=3,
                    batch_size=50,
                    validation_split = 0.1
                    )

Epoch 1/3
869/869 [==============================] - 1258s 1s/step - loss: 3.8044 - accuracy: 0.1328 - f1: 0.0525 - val_loss: 4.2752 - val_accuracy: 0.1057 - val_f1: 0.0452
Epoch 2/3
110/869 [==>...........................] - ETA: 17:25 - loss: 3.7313 - accuracy: 0.1483 - f1: 0.0569